### Softmax Regression

An extention of logistic regression for non-binary classifcation is called softmax/multinomial regression. This can also be seen as a one layer neural network in and of itself.

**MNIST** can be classifed from this methodology. 

The basic setup for softmax regression is as follows: